<a href="https://colab.research.google.com/github/NahuelRepetto/Programacion-Concurrente/blob/main/TrabajoN1Parte4Cpp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [147]:
%%writefile Sumador.cpp
#include<iostream>
#include<boost/lexical_cast.hpp>
#include<thread>
#include<mutex>

using namespace std;

int suma = 0;
int* vec;
mutex mtx;

void sumarRango(int inicio, int fin)
{      
   int subTotal = 0;
   for(int i = inicio; i < fin; i++)      
     subTotal+= vec[i];   
   mtx.lock();
   suma+= subTotal;
   mtx.unlock();
}

int main (int argc, char *argv[])
{		
	//VERIFICO QUE SE HAYAN ENVIADO LOS NECESARIOS
  if( argc < 3 ) 
  {
    cout<<"Se deben ingresar N+1 cantidad de numeros, separados con espacios. Los primeros N numeros constituiran el array, mientras que el ultimo indicará cuantos numeros tomara el primer hilo, mientras que el resto sera para el segundo, por lo que : 0 <= ultimoNumero <= N."<< endl;
    exit(1);
  }

	//VALIDO QUE LOS PARAMENTROS INGRESADOS SEAN NUMEROS Y LOS GUARDO EN UN ARRAY	
  int cantNumeros = argc -2; 
	int numeros [cantNumeros];
  int cantNumPrimerHilo;  
  string lectura;     

  for(int i = 1; i <= cantNumeros; i++)
  {
    lectura = argv[i];
    try 
		{		      
      numeros[i-1] = boost::lexical_cast<int>(lectura);      
    }
    catch (boost::bad_lexical_cast const &e) //SI LA ENTRADA ES INCORRECTA (NO NUMERICA)
		{        
      cout << "Entrada invalida '" << lectura <<"', deben ingresarse solo numeros." << endl;
			exit(1);
    }
  }

  vec = numeros;

  //VALIDO LA CANTIDAD DE NUMEROS PARA EL PRIMER HILO SEA MAYOR O IGUAL A 0 Y MENOR O IGUAL AL TOTAL DE NUMEROS	
  lectura = argv[cantNumeros+1];
  try 
	{		
    cantNumPrimerHilo = boost::lexical_cast<int>(lectura); 
    if (cantNumPrimerHilo < 0 || cantNumPrimerHilo > cantNumeros)
    {        
      cout << "Entrada invalida, la ultima entrada indicará cuantos numeros tomara el primer hilo, por lo que debe ser mayor o igual a 0 y menor o igual a la cantidad de numeros ingresados."<< endl;
		  exit(1);
    }
  }
  catch (boost::bad_lexical_cast const &e) //SI LA ENTRADA ES INCORRECTA (NO NUMERICA)
	{        
    cout << "Entrada invalida '" << lectura <<"', deben ingresarse solo numeros." << endl;
	  exit(1);
  }

  //REALIZO LA SUMA DEL CONTENIDO DEL ARRAY                
  thread HiloA (sumarRango, 0, cantNumPrimerHilo);
  thread HiloB (sumarRango, cantNumPrimerHilo, cantNumeros);
  HiloA.join();
  HiloB.join();  
    	
	//MUESTRO RESULTADOS
  cout << "Valores recibidos: ";  
  for(int i = 0; i < cantNumeros; i++)  
    cout << vec[i] << " ";  
  cout << endl << "Suma total: " << suma;  		
	return EXIT_SUCCESS;
}

Overwriting Sumador.cpp


In [145]:
!g++ -pthread -o Sumador Sumador.cpp

Se deben ingresar N+1 cantidad de numeros, separados con espacios.
Los primeros N numeros constituiran el array, mientras que el ultimo indicará cuantos numeros tomara el primer hilo, mientras que el resto sera para el segundo, por lo que : 0 <= ultimoNumero <= N

In [146]:
!./Sumador 1 2 3 -4 5 6 7 8 9 10 5


Valores recibidos: 1 2 3 -4 5 6 7 8 9 10 
Suma total: 47